# Atari Breakout in RLlib

In [1]:
#!pip install ray[rllib]
#!pip install ipywidgets gputil
#!pip install gym[atari] autorom[accept-rom-license] ray[rllib] torch torchvision atari-py ale-py ale_py
#!pip install "gym[accept-rom-license, atari]"


In [ ]:
import gym
import ray
from ray import tune
import ale_py



if ray.is_initialized():
    ray.shutdown()
ray.init(num_cpus=20)
    #env = gym.make("ALE/Breakout-v5")

checkpoint_directory = '/home/fitz/ray_results/PPO_2024-04-24_22-45-18/PPO_ALE_Breakout-v5_ff415_00000_0_2024-04-24_22-45-18/checkpoint_000004'
config = {
    "env": "ALE/Breakout-v5",
    "num_workers": 19,  
    "framework": "torch",  
    "gamma": 0.99,
    "train_batch_size": 5000,
    "rollout_fragment_length": "auto",
    "sgd_minibatch_size": 500,
    "num_sgd_iter": 10,
    "clip_param": 0.2,
    "entropy_coeff": 0.01,
}

stop_criteria = {
    "training_iteration": 300,
    "episode_reward_mean": 200
}

results = tune.run(
    "PPO",
    config=config,
    stop=stop_criteria,
    checkpoint_at_end=True,
    checkpoint_freq=5,
    keep_checkpoints_num=10,  # Keep 10 checkpoints
    checkpoint_score_attr="episode_reward_mean",  # Retain checkpoints with the highest episode rewards
)
    


2024-04-24 22:54:46,451	INFO worker.py:1749 -- Started a local Ray instance.
2024-04-24 22:54:46,996	INFO tune.py:624 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(RolloutWorker pid=104524) A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7)
(RolloutWorker pid=104524) [Powered by Stella]
(PPO pid=104365) 2024-04-24 22:55:07,681	WARNING deprecation.py:50 -- DeprecationWarning: `ray.rllib.execution.train_ops.multi_gpu_train_one_step` has been deprecated. This will raise an error in the future!
(PPO pid=104365) A.L.E: Arcade Learning Environment (version 0.8.1+53f58b7) [repeated 19x across cluster]
(PPO pid=104365) [Powered by Stella] [repeated 19x across cluster]


Trial name,agent_timesteps_total,connector_metrics,counters,custom_metrics,episode_len_mean,episode_media,episode_reward_max,episode_reward_mean,episode_reward_min,episodes_this_iter,episodes_timesteps_total,info,num_agent_steps_sampled,num_agent_steps_trained,num_env_steps_sampled,num_env_steps_sampled_this_iter,num_env_steps_sampled_throughput_per_sec,num_env_steps_trained,num_env_steps_trained_this_iter,num_env_steps_trained_throughput_per_sec,num_faulty_episodes,num_healthy_workers,num_in_flight_async_reqs,num_remote_worker_restarts,num_steps_trained_this_iter,perf,policy_reward_max,policy_reward_mean,policy_reward_min,sampler_perf,sampler_results,timers
PPO_ALE_Breakout-v5_52587_00000,140000,{},"{'num_env_steps_sampled': 140000, 'num_env_steps_trained': 140000, 'num_agent_steps_sampled': 140000, 'num_agent_steps_trained': 140000}",{},233.48,{},4,2.23,0,28,23348,"{'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 1.2862201324105262, 'cur_kl_coeff': 9.765624999999998e-05, 'cur_lr': 5e-05, 'total_loss': 0.01019454746725387, 'policy_loss': -0.0024429959803819656, 'vf_loss': 0.023546788645908236, 'vf_explained_var': 0.9033010703325272, 'kl': 0.00874496490592719, 'entropy': 1.091010309457779, 'entropy_coeff': 0.009999999999999998}, 'model': {}, 'custom_metrics': {}, 'num_agent_steps_trained': 500.0, 'num_grad_updates_lifetime': 2750.5, 'diff_num_grad_updates_vs_sampler_policy': 49.5}}, 'num_env_steps_sampled': 140000, 'num_env_steps_trained': 140000, 'num_agent_steps_sampled': 140000, 'num_agent_steps_trained': 140000}",140000,140000,140000,5000,64.7339,140000,5000,64.7339,0,19,0,0,5000,"{'cpu_util_percent': 19.645192307692305, 'ram_util_percent': 15.218269230769234, 'gpu_util_percent0': 0.1782692307692308, 'vram_util_percent0': 0.05781250000000001}",{},{},{},"{'mean_raw_obs_processing_ms': 2.163045805872632, 'mean_inference_ms': 37.33256771678569, 'mean_action_processing_ms': 0.3599760439198613, 'mean_env_wait_ms': 4.812104832964616, 'mean_env_render_ms': 0.0}","{'episode_reward_max': 4.0, 'episode_reward_min': 0.0, 'episode_reward_mean': 2.23, 'episode_len_mean': 233.48, 'episode_media': {}, 'episodes_this_iter': 28, 'episodes_timesteps_total': 23348, 'policy_reward_min': {}, 'policy_reward_max': {}, 'policy_reward_mean': {}, 'custom_metrics': {}, 'hist_stats': {'episode_reward': [2.0, 2.0, 2.0, 4.0, 2.0, 2.0, 3.0, 3.0, 2.0, 2.0, 2.0, 3.0, 2.0, 4.0, 4.0, 1.0, 2.0, 0.0, 4.0, 3.0, 2.0, 3.0, 3.0, 3.0, 2.0, 0.0, 2.0, 3.0, 4.0, 4.0, 0.0, 2.0, 4.0, 2.0, 2.0, 2.0, 2.0, 0.0, 4.0, 2.0, 3.0, 2.0, 4.0, 2.0, 4.0, 2.0, 4.0, 0.0, 4.0, 0.0, 0.0, 0.0, 3.0, 2.0, 3.0, 3.0, 0.0, 2.0, 3.0, 0.0, 3.0, 0.0, 2.0, 4.0, 4.0, 2.0, 2.0, 2.0, 2.0, 2.0, 3.0, 2.0, 3.0, 2.0, 0.0, 2.0, 3.0, 3.0, 4.0, 3.0, 4.0, 3.0, 0.0, 3.0, 2.0, 3.0, 2.0, 2.0, 2.0, 0.0, 2.0, 0.0, 3.0, 1.0, 2.0, 4.0, 0.0, 0.0, 4.0, 2.0], 'episode_lengths': [216, 232, 222, 308, 199, 227, 266, 266, 225, 211, 212, 270, 203, 325, 314, 179, 219, 135, 304, 265, 234, 274, 276, 256, 227, 155, 223, 258, 329, 302, 130, 234, 310, 225, 204, 205, 225, 156, 300, 228, 270, 207, 303, 209, 333, 218, 329, 145, 310, 146, 143, 132, 279, 214, 255, 281, 131, 225, 279, 154, 263, 148, 204, 307, 313, 230, 217, 208, 223, 222, 257, 211, 271, 211, 138, 219, 276, 263, 319, 261, 324, 275, 144, 263, 230, 266, 217, 227, 205, 150, 213, 143, 280, 159, 215, 328, 145, 137, 321, 233]}, 'sampler_perf': {'mean_raw_obs_processing_ms': 2.163045805872632, 'mean_inference_ms': 37.33256771678569, 'mean_action_processing_ms': 0.3599760439198613, 'mean_env_wait_ms': 4.812104832964616, 'mean_env_render_ms': 0.0}, 'num_faulty_episodes': 0, 'connector_metrics': {}}","{'training_iteration_time_ms': 77620.642, 'restore_workers_time_ms': 0.018, 'training_step_time_ms': 77620.596, 'sample_time_ms': 12297.427, 'load_time_ms': 1.531, 'load_throughput': 3265777.999, 'learn_time_ms': 65298.441, 'learn_throughput': 76.572, 'synch_weights_time_ms': 22.961}"


(PPO pid=104365) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/fitz/ray_results/PPO_2024-04-24_22-54-47/PPO_ALE_Breakout-v5_52587_00000_0_2024-04-24_22-54-47/checkpoint_000000)
(PPO pid=104365) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/fitz/ray_results/PPO_2024-04-24_22-54-47/PPO_ALE_Breakout-v5_52587_00000_0_2024-04-24_22-54-47/checkpoint_000001)
(PPO pid=104365) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/fitz/ray_results/PPO_2024-04-24_22-54-47/PPO_ALE_Breakout-v5_52587_00000_0_2024-04-24_22-54-47/checkpoint_000002)
(PPO pid=104365) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/fitz/ray_results/PPO_2024-04-24_22-54-47/PPO_ALE_Breakout-v5_52587_00000_0_2024-04-24_22-54-47/checkpoint_000003)
(PPO pid=104365) Checkpoint successfully created at: Checkpoint(filesystem=local, path=/home/fitz/ray_results/PPO_2024-04-24_22-54-47/PPO_ALE_Breakout-v5_52587_00000_0_2024

In [6]:
best_trial = results.get_best_trial("episode_reward_mean", "max", "last")
best_checkpoint = results.get_best_checkpoint(best_trial, "episode_reward_mean", "max")
print("Best checkpoint:", best_checkpoint)


Best checkpoint: Checkpoint(filesystem=local, path=/home/fitz/ray_results/PPO_2024-04-24_22-45-18/PPO_ALE_Breakout-v5_ff415_00000_0_2024-04-24_22-45-18/checkpoint_000004)


In [4]:
ray.shutdown()